In [24]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.metrics import Metric

from utils import *

In [4]:
train_dataset = FlowDataset(train=True)
valid_dataset = FlowDataset(train=False)

In [5]:
x_train, y_train = train_dataset.get_xy()
x_valid, y_valid = valid_dataset.get_xy()

In [6]:
feature_names = train_dataset.get_feature_names()

In [17]:
ordinal_cols = train_dataset.df.nunique()
ordinal_cols = ordinal_cols[ordinal_cols < 100].to_dict()
del ordinal_cols['fwd_seg_size_min']

In [18]:
cat_idxs = [idx for idx in range(len(feature_names)) if feature_names[idx] in ordinal_cols]
cat_dims = [ordinal_cols[i] for i in feature_names if i in ordinal_cols]

In [21]:
model = TabNetClassifier(cat_idxs=cat_idxs, cat_dims=cat_dims)

Device used : cuda


In [ ]:
class F1(Metric):
    def __init__(self) -> None:
        super().__init__()
        self._name = 'f1-score'
        self._maximize = True

    def __call__(self, y_true, y_score):
        y_pred = np.argmax(y_score, axis=1)
        return f1_score(y_true, y_pred)

In [23]:
model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], batch_size=128, num_workers=2, )

epoch 0  | loss: 0.16509 | val_0_auc: 0.9536  |  0:04:39s


KeyboardInterrupt: 